In [ ]:
import pandas as pd

df = pd.read_csv("./data/stage_3_feature_importance.csv")
df_gt = pd.read_csv("./data/stage_3_ground_truth.csv")

df_lbl = df[df.categoryId != 0]
df_unlbl = df[df.categoryId == 0]

In [ ]:
from matplotlib import pyplot as plt
from collections import Counter


def plot_label_distribution(df: pd.DataFrame, name: str = ' '):
    c = Counter()
    c.update(df['categoryId'])

    print(name)
    plt.figure(figsize=(10, 10))
    plt.pie(c.values(), labels=c.keys(), autopct='%1.1f%%')
    plt.show()


plot_label_distribution(df, 'Dataset label distribution:')
plot_label_distribution(df_gt, 'Ground-truth label distribution:')
plot_label_distribution(df_lbl, 'Dataset w/o 0 label distribution:')

In [ ]:
from imblearn.over_sampling import RandomOverSampler

y_gt = df_gt['categoryId']

y_unlbl = df_unlbl['categoryId'].replace(0, -1)
X_unlbl = df_unlbl.drop('categoryId', axis=1)

y_lbl = df_lbl['categoryId']
X_lbl = df_lbl.drop('categoryId', axis=1)

X_lbl_res, y_lbl_res = RandomOverSampler(random_state=0).fit_resample(X_lbl, y_lbl)

In [ ]:
from sklearn.model_selection import train_test_split

y_mixed = pd.concat([y_lbl_res, y_unlbl])
X_mixed = pd.concat([X_lbl_res, X_unlbl])

unlbl_indx = y_mixed[y_mixed == -1].index

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_mixed.shape, y_mixed.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X_lbl_res, y_lbl_res)
y_pred = clf.predict(X_unlbl)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelPropagation

# model = LabelPropagation(n_jobs=-1)
# model.fit(X, y)
#
# # y_pred = clf.transduction_[unlbl_indx]
# y_pred = model.predict(y_unlbl)

print(accuracy_score(y_gt[df_unlbl.index], y_pred))